In [1]:
import pandas as pd


In [2]:
golden_df = pd.read_csv("Resources/golden_globe_awards.csv")
oscar_df = pd.read_csv("Resources/the_oscar_award.csv")



# Golden Globes Information


In [3]:
golden_df.head()

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


# Oscar Information

In [13]:
oscar_df.head()

oscar_df = oscar_df.rename(columns={'name': 'nominee'})
oscar_df.head()

,year_film,year_ceremony,ceremony,category,nominee,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


# Film Dataframe


In [17]:
film_df = golden_df.merge(oscar_df, how = "outer", on='film', suffixes=('_golden', '_oscar'))

film_df.head()

,year_film_golden,year_award,ceremony_golden,category_golden,nominee_golden,film,win,year_film_oscar,year_ceremony,ceremony_oscar,category_oscar,nominee_oscar,winner
0,1943.0,1944.0,1.0,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True,NaN,NaN,NaN,NaN,NaN,NaN
1,1943.0,1944.0,1.0,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True,NaN,NaN,NaN,NaN,NaN,NaN
2,1943.0,1944.0,1.0,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True,NaN,NaN,NaN,NaN,NaN,NaN
3,1943.0,1944.0,1.0,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True,NaN,NaN,NaN,NaN,NaN,NaN
4,1943.0,1944.0,1.0,Picture,The Song Of Bernadette,NaN,True,1927.0,1928.0,1.0,ENGINEERING EFFECTS,Ralph Hammeras,False


In [19]:
#had we had more time we would have added a separate collection dedicated actors 


#actor_df = golden_df.merge(oscar_df, how = "outer", on='nominee', suffixes=('_golden', '_oscar'))
#actor_df.head()

,year_film_golden,year_award,ceremony_golden,category_golden,nominee,film_golden,win,year_film_oscar,year_ceremony,ceremony_oscar,category_oscar,film_oscar,winner
0,1943.0,1944.0,1.0,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True,1943.0,1944.0,16.0,ACTRESS IN A SUPPORTING ROLE,For Whom the Bell Tolls,True
1,1943.0,1944.0,1.0,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True,1936.0,1937.0,9.0,ACTOR IN A SUPPORTING ROLE,The General Died at Dawn,False
2,1943.0,1944.0,1.0,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True,1943.0,1944.0,16.0,ACTOR IN A SUPPORTING ROLE,For Whom the Bell Tolls,False
3,1943.0,1944.0,1.0,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True,1943.0,1944.0,16.0,DIRECTING,The Song of Bernadette,False
4,1943.0,1944.0,1.0,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True,1944.0,1945.0,17.0,DIRECTING,Wilson,False


In [21]:
unique_film = film_df['film'].unique()

len(unique_film)

6975

In [22]:
post={
    'film' : '',
    'year' : '',
    'oscar_nominations' : [],
    'golden_nominations' : [],
    
    
}



# Creating the Mongo Database


In [41]:
import pymongo 

In [42]:
connection = "mongodb://localhost:27017"
client = pymongo.MongoClient(connection)

In [44]:
db = client.award_db
db.film.drop()
film_collection = db.film


In [47]:
#counter 
i=0



#looped through the unique films and created lists for each award type

for film in unique_film:
    
    #filter by film 
    
    df = film_df[film_df['film'] == film]
    
    #creating a dictionary to add results to 
    
    post={
    'film' : film,
    'oscar_nominations' : [],
    'golden_nominations' : [],
    
    
    }
    
    #making a filtered dataframe for the gloden globe information to create a dictionary
    
    golden_nominations = df['category_golden'].unique()
    for nomination in golden_nominations:
        tf = df[df['category_golden'] == nomination]
        
        #knowing that there will be blanks in the golden globe data we put in error handling so that the loop would not error out 
        #for when a blank was hit 
        
        try:
            
            cat_nom = {
                'year_film' : tf['year_film_golden'].iloc[0],
                'year_award' : tf['year_award'].iloc[0],
                'category' : tf['category_golden'].iloc[0] ,
                'nominee' : tf['nominee_golden'].iloc[0] ,
                'win' : tf['win'].iloc[0] ,
            }
            post['golden_nominations'].append(cat_nom)
        except:
            pass
        
        
      
    #making a filtered dataframe for the oscars information to create a dictionary
        
    oscar_nominations = df['category_oscar'].unique()
    for nomination in oscar_nominations:
        tf = df[df['category_oscar'] == nomination]
        
          
        #knowing that there will be blanks in the oscar data we put in error handling so that the loop would not error out 
        #for when a blank was hit 
        
        try:
            
            cat_nom = {
                'year_film' : tf['year_film_oscar'].iloc[0],
                'year_award' : tf['year_ceremony'].iloc[0],
                'category' : tf['category_oscar'].iloc[0] ,
                'nominee' : tf['nominee_oscar'].iloc[0] ,
                'win' : tf['winner'].iloc[0] ,
            }
            post['oscar_nominations'].append(cat_nom)
        
        except:
            pass
    
    
#picked a breakpoint to test the code and troubleshoot
    #i=i+1
    #if i == 5:
        #break
    
    id=film_collection.insert_one(post).inserted_id

In [37]:
#showing an example of the data pulled from the CSV's and how it looks in the mongo database 

post

{'film': 'Mrs. Parkington',
 'oscar_nominations': [{'year_film': 1944.0,
   'year_award': 1945.0,
   'category': 'ACTRESS',
   'nominee': 'Greer Garson',
   'win': False},
  {'year_film': 1944.0,
   'year_award': 1945.0,
   'category': 'ACTRESS IN A SUPPORTING ROLE',
   'nominee': 'Agnes Moorehead',
   'win': False}],
 'golden_nominations': [{'year_film': 1944.0,
   'year_award': 1945.0,
   'category': 'Best Performance by an Actress in a Supporting Role in any Motion Picture',
   'nominee': 'Agnes Moorehead',
   'win': True}]}